In [ ]:
'''
IAPM Portfolio discovery
Bharath, Chelsea, Gaurav, Vikram, Utkarsh, Yash

2009-2021 dataset link: https://drive.google.com/file/d/1yJuY2SF3sCllibGXeyoNNzS6v2j7g08s/view?usp=sharing
01/08/24-31/12/25 dataset link: https://drive.google.com/file/d/17DOkvtHBUXqDMvnWVTku1TfRFNEKolS7/view?usp=sharing
01/08/24-25/02/26 dataset link: https://drive.google.com/file/d/1edQni7obD5aQqfgqh_-bgz0NOOj6tXG7/view?usp=sharing
'''

'\nIAPM Portfolio discovery\nBharath, Chelsea, Gaurav, Vikram, Utkarsh, Yash\n\n2009-2021 dataset link: https://drive.google.com/file/d/1yJuY2SF3sCllibGXeyoNNzS6v2j7g08s/view?usp=sharing\n01/08/24-31/12/25 dataset link: https://drive.google.com/file/d/17DOkvtHBUXqDMvnWVTku1TfRFNEKolS7/view?usp=sharing\n01/08/24-25/02/26 dataset link: https://drive.google.com/file/d/1edQni7obD5aQqfgqh_-bgz0NOOj6tXG7/view?usp=sharing\n'

In [3]:
import os
import io
import zipfile
import requests
import itertools
import numpy as np
import pandas as pd
import gdown
import glob
import math

NIFTY50_SYMBOLS = [
    "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY",
    "EICHERMOT", "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE",
    "HEROMOTOCO", "HINDALCO", "HINDUNILVR", "ICICIBANK", "ITC",
    "INDUSINDBK", "INFY", "JSWSTEEL", "KOTAKBANK", "LT",
    "LTIM", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN",
    "SUNPHARMA", "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL",
    "TECHM", "TITAN", "ULTRACEMCO", "UPL", "WIPRO"
]

CHELSEA01 = ["ASLIND", "TAKE", "BANCOINDIA","KESORAMIND", "BEL", "LOTUSEYE", "AXISILVER","CRAFTSMAN",
             "CUB", "QGOLDHALF", "SILVER", "KAPSTON", "SABEVENTS", "TDPOWERSYS", "BLISSGVS", "SANSERA",
             "LUMAXIND", "MRPL", "PRECWIRE", "EGOLD", "KRISHANA", "NETWEB", "BGRENERGY", "HAPPYFORGE", "GENCON",
             "CMMIPL", "AVANTIFEED", "BHARATWIRE", "GVT&D", "POWERINDIA", "LTF", "ABINFRA", "UNIONBANK", "BANKINDIA",
             "CUPID", "SMSPHARMA", "UNIHEALTH", "VMARCIND", "TFCILTD", "DCBBANK", "KARURVYSYA", "TCIEXP"]

Bharath = ["ACUTAAS", "PARKHOTELS", "APOLLOHOSP", "BAJAJ-AUTO", "CANROBO",
       "CAMS", "CONTROLPR", "CROMPTON", "DABUR", "GRSE",
       "MEDANTA", "HEG", "KOTAKBANK", "LT", "LICHSGFIN",
       "REPCOHOME", "SAILIFE", "SKYGOLD", "SPAL", "SBIN",
       "V2RETAIL", "WABAG", "VBL"]

COMPANY_STARTING_WITH_A_SYMBOLS = [
    "A2ZMES", "AANJANEYA", "AARTIDRUGS", "AARTIIND", "AARVEEDEN", "ABAN",
    "ABB", "ABCIL", "ABGSHIP", "ABIRLANUVO", "ACC", "ACE", "ACKRUTI", "ADANIENT",
    "ACROPETAL", "ADANIPOWER", "ADFFOODS", "ADHUNIK", "ADORWELD", "ADSL",
    "ADVANIHOTR", "ADVANTA", "AEGISCHEM", "AFL", "AFTEK", "AGCNET", "AGRODUTCH",
    "AHLEAST", "AHLUCONT", "AHLWEST", "AHMEDFORGE", "AIAENG", "AICHAMP", "AJANTPHARM",
    "AJMERA", "AKSHOPTFBR", "AKZOINDIA", "ALBK", "ALCHEM", "ALEMBICLTD",
    "ALFALAVAL", "ALICON", "ALKALI", "ALKYLAMINE", "ALLCARGO", "ALLSEC",
    "ALMONDZ", "ALOKTEXT", "ALPHAGEO", "AMAR", "AMARAJABAT", "ALPSINDUS", "AMBIKCO",
    "AMBUJACEM", "AMDIND", "AMTEKAUTO", "AMTEKINDIA", "ANANTRAJ", "ANDHRABANK",
    "ANDHRSUGAR", "ANGIND", "ANIKINDS", "ANDHRACEMT", "ANKURDRUGS", "ANSALAPI",
    "ANSALHSG", "ANTGRAPHIC", "APARINDS", "APCOTEXIND", "APIL", "APOLLOHOSP",
    "APOLLOTYRE", "APPAPER", "APTECHT", "AQUA", "ARCHIDPLY", "ARCHIES", "AREVAT&D"
]

UTKARSH_TICKERS = ["OSIAJEE",
"GOLDETF",
"CIANAGRO",
"GNRL",
"CUPID",
"INDOKEM",
"ARIHANT",
"LKPFIN",
"BGRENERGY",
"JSWHL"]

def find_data_root(base_folder):
    """
    Recursively search for Companies_list.csv to detect valid data root.
    """
    for root, dirs, files in os.walk(base_folder):
        if "Companies_list.csv" in files and "HISTORICAL_DATA" in dirs:
            return root
    return None

def download_and_extract_google_drive_zip_1(file_id, extract_to="stock_data"):
    """
    Downloads and extracts Google Drive ZIP safely.
    Automatically detects correct nested root folder.
    """
    os.makedirs(extract_to, exist_ok=True)
    existing_root = find_data_root(extract_to)
    if existing_root is not None:
        print(f"Data already exists at: {existing_root}")
        print("Skipping download.")
        return existing_root
    zip_path = os.path.join(extract_to, "stock_data.zip")
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Downloading from Google Drive...")
    gdown.download(url, zip_path, quiet=False)
    print("Extracting zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction complete.")
    for root, dirs, files in os.walk(extract_to): # recursively find Companies_list.csv
        if "Companies_list.csv" in files:
            print("Correct data root detected:", root)
            return root
    raise Exception("Could not find Companies_list.csv in extracted contents")

def load_company_csv_strict(path: str, symbol: str) -> pd.DataFrame | None:
    """
    Load single CSV only if it contains 'Date' and a close-like column.
    """

    try:
        raw = pd.read_csv(path, low_memory=False)
        raw.columns = [str(c).strip() for c in raw.columns]
        # find exact date column (case-insensitive)
        date_col = next((c for c in raw.columns if c.lower() == "date"), None)
        if date_col is None:
            # skip
            # print(f"Skipping {symbol}: no Date column")
            return None
        raw[date_col] = pd.to_datetime(raw[date_col], errors="coerce")
        raw = raw.dropna(subset=[date_col])
        close_candidates = [c for c in raw.columns if c.lower() in ("adj_close","adj close","close","close_price","close price")]
        if not close_candidates:
            # skip
            # print(f"Skipping {symbol}: no close column")
            return None
        close_col = close_candidates[0]
        df = raw[[date_col, close_col]].rename(columns={date_col: "Date", close_col: symbol})
        df = df.dropna().drop_duplicates(subset=["Date"]).sort_values("Date").set_index("Date")
        return df
    except Exception as e:
        print(f"Error loading {symbol} from {path}: {e}")
        return None

def read_stock_data_from_folder(folder_path):
    """
    Reads stock dataset using:

    folder_path/
        Companies_list.csv
        HISTORICAL_DATA/*.csv

    Returns optimizer-compatible dataframe:
    date, SYMBOL1, SYMBOL2, SYMBOL3...
    """

    companies_csv = os.path.join(folder_path, "Companies_list.csv")
    hist_dir = os.path.join(folder_path, "HISTORICAL_DATA")
    if not os.path.exists(companies_csv):
        raise Exception("Companies_list.csv not found")
    if not os.path.exists(hist_dir):
        raise Exception("HISTORICAL_DATA folder not found")
    companies_df = pd.read_csv(companies_csv)
    companies_df.columns = [c.strip() for c in companies_df.columns]
    symbol_col = next(
        (c for c in companies_df.columns
         if c.lower() in ("symbol", "ticker", "company", "stock", "code")),
        companies_df.columns[0]
    )
    symbols = companies_df[symbol_col].astype(str).tolist()
    files = glob.glob(os.path.join(hist_dir, "*_data.csv"))
    file_map = {}
    for f in files:
        name = os.path.basename(f)
        prefix = name.replace("_data.csv", "")
        file_map[prefix.upper()] = f
    frames = []
    loaded_symbols = []
    for symbol in symbols:
        symbol_upper = symbol.upper()
        if symbol_upper not in file_map:
            continue
        df = load_company_csv_strict(file_map[symbol_upper], symbol)
        if df is not None and not df.empty:
            frames.append(df)
            loaded_symbols.append(symbol)
    if not frames:
        raise Exception("No valid CSV files found after strict cleaning")
    panel = pd.concat(frames, axis=1, join="outer")
    panel = panel.sort_index().ffill()
    # convert to optimizer format
    panel.reset_index(inplace=True)
    panel.rename(columns={"Date": "date"}, inplace=True)
    panel["date"] = panel["date"].astype(str)
    print(f"Loaded {len(loaded_symbols)} valid symbols")
    return panel

def download_and_extract_google_drive_zip_2(file_id, extract_to="bse_data"):
    os.makedirs(extract_to, exist_ok=True)
    zip_path = os.path.join(extract_to, "stock_data_24_26_25Feb26.zip")
    url = f"https://drive.google.com/uc?id={file_id}"
    if not os.path.exists(zip_path):
        print("Downloading ZIP from Google Drive...")
        gdown.download(url, zip_path, quiet=False)
    print("Extracting ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction complete.")
    # Return actual CSV folder directly
    csv_folder = os.path.join(extract_to, "stock_data_24_26_25Feb26")
    return csv_folder

def select_top_by_turnover(df, top_n=100):
    avg_turnover = (
        df.groupby("symbol")["turnover"]
        .mean()
        .sort_values(ascending=False)
    )
    return avg_turnover.head(top_n).index.tolist()

def select_random(df, top_n=100, seed=42):
    np.random.seed(seed)
    symbols = df["symbol"].unique()
    return list(np.random.choice(symbols, size=min(top_n, len(symbols)), replace=False))

def select_price_filter(df, min_price=50):
    avg_price = df.groupby("symbol")["close"].mean()
    return avg_price[avg_price > min_price].index.tolist()

def select_turnover_and_price(df, top_n=150, min_price=50):
    df_filtered = df[df["close"] > min_price]
    return select_top_by_turnover(df_filtered, top_n=top_n)

def read_bse_bhavcopy_folder(
    folder_path,
    strategy=None,
    strategy_params=None
):
    """
    strategy:
        "top_turnover"
        "random"
        "price_filter"
        "turnover_price"
        None  → no filtering (ALL stocks)
    returns data in
    Date firm1 firm2...
    ...  price price...
    """
    if strategy_params is None:
        strategy_params = {}
    print("Scanning CSV files...")
    csv_files = glob.glob(os.path.join(folder_path, "**/*.[cC][sS][vV]"), recursive=True)
    if not csv_files:
        raise Exception("No CSV files found.")
    all_data = []
    for file in csv_files:
        try:
            df = pd.read_csv(file, low_memory=False)
            required_cols = ["TradDt", "TckrSymb", "ClsPric", "TtlTrfVal"]
            if not all(col in df.columns for col in required_cols):
                continue
            df = df[required_cols].copy()
            df["TradDt"] = pd.to_datetime(df["TradDt"], errors="coerce")
            df["TtlTrfVal"] = pd.to_numeric(df["TtlTrfVal"], errors="coerce")
            df = df.dropna(subset=["TradDt", "TckrSymb", "ClsPric", "TtlTrfVal"])
            df.rename(columns={
                "TradDt": "date",
                "TckrSymb": "symbol",
                "ClsPric": "close",
                "TtlTrfVal": "turnover"
            }, inplace=True)

            all_data.append(df)
        except:
            continue
    if not all_data:
        raise Exception("No valid bhav copy data found.")
    combined = pd.concat(all_data, ignore_index=True)
    # Strategy Selection
    if strategy is None:
      print("Strategy: None (returning ALL stocks)")
      selected_symbols = combined["symbol"].unique().tolist()
    elif strategy == "top_turnover":
        selected_symbols = select_top_by_turnover(combined, **strategy_params)
    elif strategy == "random":
        selected_symbols = select_random(combined, **strategy_params)
    elif strategy == "price_filter":
        selected_symbols = select_price_filter(combined, **strategy_params)
    elif strategy == "turnover_price":
        selected_symbols = select_turnover_and_price(combined, **strategy_params)
    elif strategy is None:
        selected_symbols = combined["symbol"].unique().tolist()
    else:
        raise ValueError("Unknown strategy")
    print(f"Strategy: {strategy}")
    print(f"Selected {len(selected_symbols)} stocks")
    combined = combined[combined["symbol"].isin(selected_symbols)]
    # Pivot
    panel = combined.pivot_table(
        index="date",
        columns="symbol",
        values="close",
        aggfunc="first"
    )
    panel = panel.sort_index().ffill()
    panel.reset_index(inplace=True)
    panel["date"] = panel["date"].astype(str)
    print(f"Final dataset shape: {panel.shape}")
    return panel

def filter_dataset(
    df,
    start_date=None,
    end_date=None,
    company_codes=None
):
    """
    Filters dataset based on tunable parameters...

    INPUTS:
    df            : pandas DataFrame dataset in specified format
    start_date    : str or None starting date to include
    end_date      : str or None ending date to include
    company_codes : list[str] or None list of company columns to keep

    OUTPUT:
    filtered_df   : pandas DataFrame

    INPUT DATAFRAME FORMAT:
    First column      : date (string / datetime / month)
    Remaining columns : stock prices
    Each row          : time snapshot
    Each column       : stock price of that company at that time
    example           :
    date,    AAPL, MSFT, GOOG, AMZN
    2020-01, 100, 200, 300, 400
    2020-02, 105, 210, 290, 420
    2020-03, 110, 220, 310, 430
    """

    filtered = df.copy()
    filtered['date'] = pd.to_datetime(filtered['date']) # Ensure date column is datetime for comparison
    if start_date is not None: # Filter by start date
        filtered = filtered[filtered['date'] >= pd.to_datetime(start_date)]
    if end_date is not None: # Filter by end date
        filtered = filtered[filtered['date'] <= pd.to_datetime(end_date)]
    if company_codes is not None: # Filter company columns
        filtered = filtered[['date'] + company_codes]
    filtered = filtered.reset_index(drop=True) # Reset index
    return filtered

def compute_returns(price_df):
    """
    Converts price data into return data...

    INPUT:
    price_df : pandas DataFrame

    RETURN DEFINTITION:
    r_t = (P_t - P_(t-1)) / P_(t-1)

    OUTPUT:
    returns_df : pandas DataFrame
    """
    prices = price_df.copy()
    prices.set_index('date', inplace=True)
    # Remove columns that contain ANY NaN in the selected window
    prices = prices.dropna(axis=1, how="any")
    returns_df = prices.pct_change().dropna()
    return returns_df

def portfolio_optimizer(returns_df, risk_free_rate=0.0, allow_short=True):
    """
    Performs closed-form portfolio optimisation.

    INPUT:
    returns_df      : pandas DataFrame
    risk_free_rate  : float

    OUTPUT:
    results         : dictionary
                        mean return vector
                        covariance matrix
                        GMVP portfolio (min variance)
                        MRR portfolio (max sharpe)
    """

    returns = returns_df.values # Convert to numpy

    # edge case: if portfolio by chance has only one stock
    if returns.shape[1] == 1:
      mu = returns.mean(axis=0)[0]
      sigma = returns.std(axis=0)[0]
      return {
          "mean_returns": np.array([mu]),
          "std_returns": np.array([sigma]),
          "covariance_matrix": np.array([[sigma**2]]),
          "gmvp": {
              "weights": np.array([1.0]),
              "return": mu,
              "risk": sigma
          },
          "mrr": {
              "weights": np.array([1.0]),
              "return": mu,
              "risk": sigma,
              "sharpe": (mu - risk_free_rate) / sigma
          }
      }

    mu = np.mean(returns, axis=0) # Mean returns vector
    Sigma = np.cov(returns, rowvar=False) # Covariance matrix
    Sigma_inv = np.linalg.inv(Sigma) # Inverse covariance
    n = len(mu)
    ones = np.ones(n)

    # GMVP Portfolio
    w_gmvp = Sigma_inv @ ones
    w_gmvp = w_gmvp / (ones.T @ Sigma_inv @ ones)
    if not allow_short: # enforce long-only constraint if short selling not allowed
      w_gmvp = np.maximum(w_gmvp, 0)
      w_gmvp = w_gmvp / np.sum(w_gmvp)
    gmvp_return = w_gmvp @ mu
    gmvp_var = w_gmvp.T @ Sigma @ w_gmvp
    gmvp_std = np.sqrt(gmvp_var)

    # MRR Portfolio
    excess_returns = mu - risk_free_rate
    w_tan = Sigma_inv @ excess_returns
    w_tan = w_tan / (ones.T @ Sigma_inv @ excess_returns)
    if not allow_short: # enforce long-only constraint if short selling not allowed
      w_tan = np.maximum(w_tan, 0)
      w_tan = w_tan / np.sum(w_tan)
    tan_return = w_tan @ mu
    tan_var = w_tan.T @ Sigma @ w_tan
    tan_std = np.sqrt(tan_var)
    tan_sharpe = (tan_return - risk_free_rate) / tan_std

    stock_std = np.sqrt(np.diag(Sigma)) # Individual stock stats
    results = {
        "mean_returns": mu,
        "std_returns": stock_std,
        "covariance_matrix": Sigma,
        "gmvp": {
            "weights": w_gmvp,
            "return": gmvp_return,
            "risk": gmvp_std
        },
        "mrr": {
            "weights": w_tan,
            "return": tan_return,
            "risk": tan_std,
            "sharpe": tan_sharpe
        }
    }
    return results

def optimized_portfolio_printer(results):
    print("\nMean returns:")
    print(results["mean_returns"])
    print("\nStd deviation:")
    print(results["std_returns"])
    print("\nGMVP weights:")
    print(results["gmvp"]["weights"])
    print("\nGMVP return:", results["gmvp"]["return"])
    print("GMVP risk:", results["gmvp"]["risk"])
    print("\nMRR weights:")
    print(results["mrr"]["weights"])
    print("\nMRR return:", results["mrr"]["return"])
    print("MRR risk:", results["mrr"]["risk"])
    print("MRR Sharpe:", results["mrr"]["sharpe"])

def BRUTE_FORCE_find_best_portfolios_of_size_x(
    filtered_df,
    x,
    risk_free_rate=0.0,
    allow_short=True
):
    """
    Try all combinations of size x.

    INPUT:
    filtered_df     : pandas DataFrame
    x               : size of portfolio
    risk_free_rate  : float

    OUTPUT:
    best_gmvp       : Best GMVP portfolio dictionary
    best_mrr        : Best MRR portfolio dictionary

    COMPLEXITY: O(n choose x) ~
    n: company pool size
    x: portfolio size
    """

    companies = list(filtered_df.columns)
    companies.remove("date")
    total_combinations = math.comb(len(companies), x)
    best_gmvp = None
    best_gmvp_risk = np.inf
    best_mrr = None
    best_mrr_sharpe = -np.inf
    processed = 0
    checked = 0
    dropped = 0
    print(f"\nTesting {len(companies)} choose {x} combinations...")
    for combo in itertools.combinations(companies, x):
        processed += 1
        sub_df = filter_dataset(filtered_df, company_codes=list(combo))
        returns = compute_returns(sub_df)
        if returns.shape[0] < 2 or returns.shape[1] < x: # ensure enough data
          dropped += 1
          continue
        Sigma = np.cov(returns.values, rowvar=False) # compute covariance matrix
        det = np.linalg.det(Sigma) # skip singular covariance matrix
        if abs(det) < 1e-12:
          dropped += 1
          continue
        checked += 1
        results = portfolio_optimizer(returns, risk_free_rate, allow_short) # safe to optimize
        gmvp_risk = results["gmvp"]["risk"]
        mrr_sharpe = results["mrr"]["sharpe"]
        # Track best GMVP
        if gmvp_risk < best_gmvp_risk:
            best_gmvp_risk = gmvp_risk
            best_gmvp = {
                "companies": combo,
                "gmvp": results["gmvp"],
                "mrr": results["mrr"]
            }

        # Track best MRR
        if mrr_sharpe > best_mrr_sharpe:
            best_mrr_sharpe = mrr_sharpe
            best_mrr = {
                "companies": combo,
                "gmvp": results["gmvp"],
                "mrr": results["mrr"]
            }

        if processed % 10000 == 0:
            print(
                f"Processed {processed}/{total_combinations} | "
                f"Checked {checked} | Dropped {dropped} | "
                f"Best GMVP risk={best_gmvp_risk:.6f}, return={best_gmvp['gmvp']['return']:.6f} | "
                f"Best MRR return={best_mrr['mrr']['return']:.6f}, risk={best_mrr['mrr']['risk']:.6f}"
            )
    print("\n===== FINAL SUMMARY =====")
    print(f"Total combinations: {total_combinations}")
    print(f"Processed: {processed}")
    print(f"Valid (checked): {checked}")
    print(f"Dropped: {dropped}")
    print(f"Drop rate: {100*dropped/processed:.2f}%")
    return best_gmvp, best_mrr


def BEAM_SEARCH_find_best_portfolios_of_size_x(
    filtered_df,
    x,
    risk_free_rate=0.0,
    allow_short=True,
    beam_width=10,
    random_samples=5000,
    sharpe_filter_size=30,
    corr_threshold=0.9 # stocks with corr > 0.9 cropped off
):
    """
    Greedy Heuristic
    Step 1: Sharpe filter: Some stocks are clearly bad: Low return, High volatility, poor sharpe: Will never appear in an optimal portfolio
            Modern portfolio theory tells us optimal portfolios lie in span of high-Sharpe assets. Low-Sharpe assets almost never contribute
    Step 2: Correlation pruning: if two stocks are highly correlated, they move almost identically, owning both adds little diversification.
    Step 3: Beam search: Instead of trying all combinations, grow portfolios step-by-step. At each step, keep only the best few candidates (=beam width)
    Step 4: Random refinement: Beam search is is greedy -> it may miss some good combinations (local oprimization) -> Explore regions beam search may miss
            Random sampling converges to global optimum as samples increase (Monte Carlo optimization principle)

    COMPLEXITY: O(knx)
    k: beam size
    n: company pool size
    x: portfolio size
    """

    companies = list(filtered_df.columns)
    companies.remove("date")
    print("\n===== STEP 1: Sharpe filtering =====")
    base_returns = compute_returns(filtered_df)
    mean = base_returns.mean()
    std = base_returns.std()
    sharpe = (mean - risk_free_rate) / std
    sharpe = sharpe.replace([np.inf, -np.inf], np.nan).dropna()
    sharpe_sorted = sharpe.sort_values(ascending=False)
    filtered_companies = list(sharpe_sorted.head(sharpe_filter_size).index)
    print(f"Reduced from {len(companies)} to {len(filtered_companies)} stocks by Sharpe")
    print("\n===== STEP 2: Correlation pruning =====")
    corr = base_returns[filtered_companies].corr().abs()
    selected_uncorr = []
    for stock in filtered_companies:
        keep = True
        for s in selected_uncorr:
            if corr.loc[stock, s] > corr_threshold:
                keep = False
                break
        if keep:
            selected_uncorr.append(stock)
    print(f"Reduced to {len(selected_uncorr)} stocks after correlation pruning")
    search_space = selected_uncorr
    total_combinations = math.comb(len(search_space), x) if len(search_space) >= x else 0
    best_gmvp = None
    best_gmvp_risk = np.inf
    best_mrr = None
    best_mrr_sharpe = -np.inf
    processed = 0
    checked = 0
    dropped = 0
    print(f"\nTesting heuristic search over reduced space...")
    print("\n===== STEP 3: Beam search =====")
    beam = [([], None)]
    for depth in range(x):
        candidates = []
        for combo, _ in beam:
            for stock in search_space:
                if stock in combo:
                    continue
                new_combo = combo + [stock]
                sub_df = filter_dataset(filtered_df, company_codes=new_combo)
                returns = compute_returns(sub_df)
                if returns.shape[1] < len(new_combo):
                    dropped += 1
                    continue
                Sigma = np.cov(returns.values, rowvar=False)
                if np.ndim(Sigma) < 2:
                    results = portfolio_optimizer(
                        returns,
                        risk_free_rate,
                        allow_short
                    )
                else:
                    if np.linalg.cond(Sigma) > 1e12:
                        dropped += 1
                        continue
                    results = portfolio_optimizer(
                        returns,
                        risk_free_rate,
                        allow_short
                    )
                sharpe_val = results["mrr"]["sharpe"]
                candidates.append((new_combo, sharpe_val, results))
                processed += 1
                checked += 1
                gmvp_risk = results["gmvp"]["risk"]
                mrr_sharpe = results["mrr"]["sharpe"]
                # Track best GMVP
                if gmvp_risk < best_gmvp_risk:
                    best_gmvp_risk = gmvp_risk
                    best_gmvp = {
                        "companies": tuple(new_combo),
                        "gmvp": results["gmvp"],
                        "mrr": results["mrr"]
                    }
                # Track best MRR
                if mrr_sharpe > best_mrr_sharpe:
                    best_mrr_sharpe = mrr_sharpe
                    best_mrr = {
                        "companies": tuple(new_combo),
                        "gmvp": results["gmvp"],
                        "mrr": results["mrr"]
                    }
        candidates.sort(key=lambda x: x[1], reverse=True)
        beam = [(c[0], c[2]) for c in candidates[:beam_width]]
        print(
            f"Depth {depth+1}/{x} | "
            f"Processed {processed} | Checked {checked} | Dropped {dropped} | "
            f"Best GMVP risk={best_gmvp_risk:.6f}, return={best_gmvp['gmvp']['return']:.6f} | "
            f"Best MRR return={best_mrr['mrr']['return']:.6f}, risk={best_mrr['mrr']['risk']:.6f}"
        )
    print("\n===== STEP 4: Random refinement =====")
    for i in range(random_samples):
        combo = tuple(np.random.choice(search_space, x, replace=True))
        sub_df = filter_dataset(filtered_df, company_codes=list(combo))
        returns = compute_returns(sub_df)
        if returns.shape[1] < x:
            dropped += 1
            continue
        Sigma = np.cov(returns.values, rowvar=False)
        if np.ndim(Sigma) < 2:
            results = portfolio_optimizer(
                returns,
                risk_free_rate,
                allow_short
            )
        else:
            if np.linalg.cond(Sigma) > 1e12:
                dropped += 1
                continue
            results = portfolio_optimizer(
                returns,
                risk_free_rate,
                allow_short
            )
        processed += 1
        checked += 1
        gmvp_risk = results["gmvp"]["risk"]
        mrr_sharpe = results["mrr"]["sharpe"]

        if gmvp_risk < best_gmvp_risk:
            best_gmvp_risk = gmvp_risk
            best_gmvp = {
                "companies": combo,
                "gmvp": results["gmvp"],
                "mrr": results["mrr"]
            }

        if mrr_sharpe > best_mrr_sharpe:
            best_mrr_sharpe = mrr_sharpe
            best_mrr = {
                "companies": combo,
                "gmvp": results["gmvp"],
                "mrr": results["mrr"]
            }

        if i % 1000 == 0 and i > 0:
            print(
                f"Random {i}/{random_samples} | "
                f"Processed {processed} | Checked {checked} | Dropped {dropped} | "
                f"Best GMVP risk={best_gmvp_risk:.6f}, return={best_gmvp['gmvp']['return']:.6f} | "
                f"Best MRR return={best_mrr['mrr']['return']:.6f}, risk={best_mrr['mrr']['risk']:.6f}"
            )
    print("\n===== FINAL SUMMARY =====")
    print(f"Total combinations (original space): {total_combinations}")
    print(f"Processed: {processed}")
    print(f"Valid (checked): {checked}")
    print(f"Dropped: {dropped}")
    if processed > 0:
        print(f"Drop rate: {100*dropped/processed:.2f}%")
    else:
        print("Drop rate: 0%")
    return best_gmvp, best_mrr


TRADING_DAYS = 252
RISK_FREE_RATE = (1 + 0.0667)**(1/TRADING_DAYS) - 1
def print_portfolio(title, portfolio):
    if portfolio is None:
        print(f"\n{title}: None")
        return
    companies = portfolio["companies"]
    gmvp = portfolio["gmvp"]
    mrr = portfolio["mrr"]
    gmvp_return_daily = gmvp["return"]
    gmvp_risk_daily = gmvp["risk"]
    gmvp_return_annual = gmvp_return_daily * TRADING_DAYS
    gmvp_risk_annual = gmvp_risk_daily * math.sqrt(TRADING_DAYS)
    mrr_return_daily = mrr["return"]
    mrr_risk_daily = mrr["risk"]
    mrr_return_annual = mrr_return_daily * TRADING_DAYS
    mrr_risk_annual = mrr_risk_daily * math.sqrt(TRADING_DAYS)
    sharpe_daily = mrr["sharpe"]
    sharpe_annual = sharpe_daily * math.sqrt(TRADING_DAYS)
    print("\n" + "="*70)
    print(f"{title}")
    print("="*70)
    print("\nCompanies:")
    for c in companies:
        print(f"  {c}")
    print("\nGMVP Portfolio Weights:")
    print("-"*40)
    for c, w in zip(companies, gmvp["weights"]):
        print(f"{c:15s} : {w*100:8.3f} %")
    print("\nGMVP Performance:")
    print(f"  Daily Return  : {gmvp_return_daily*100:8.4f} %")
    print(f"  Daily Risk    : {gmvp_risk_daily*100:8.4f} %")
    print(f"  Annual Return : {gmvp_return_annual*100:8.2f} %")
    print(f"  Annual Risk   : {gmvp_risk_annual*100:8.2f} %")
    print("\nMRR Portfolio (Max Sharpe) Weights:")
    print("-"*40)
    for c, w in zip(companies, mrr["weights"]):
        print(f"{c:15s} : {w*100:8.3f} %")
    print("\nMRR Performance:")
    print(f"  Daily Return  : {mrr_return_daily*100:8.4f} %")
    print(f"  Daily Risk    : {mrr_risk_daily*100:8.4f} %")
    print(f"  Annual Return : {mrr_return_annual*100:8.2f} %")
    print(f"  Annual Risk   : {mrr_risk_annual*100:8.2f} %")
    print(f"\nSharpe Ratio:")
    print(f"  Daily Sharpe  : {sharpe_daily:8.4f}")
    print(f"  Annual Sharpe : {sharpe_annual:8.4f}")
    print("="*70)

def symbol_exists_checker(available, symbols):
    symbols_available = []
    symbols_missing = []
    for s in symbols:
        if s in available:
            symbols_available.append(s)
        else:
            symbols_missing.append(s)
    print("Available:", len(symbols_available))
    print("Missing:", len(symbols_missing))
    print("\nMissing symbols:")
    print(symbols_missing)
    return symbols_available

def export_both_portfolios_to_excel(
    full_price_df,
    best_gmvp,
    best_mrr,
    file_name="optimized_portfolios.xlsx"
):
    """
    Exports:
    - Price table (date + portfolio stocks)
    - Portfolio weights
    - Performance metrics
    Into same Excel sheet (results appended below price table)
    """
    with pd.ExcelWriter(file_name, engine="xlsxwriter") as writer:
        # ================= GMVP =================
        if best_gmvp is not None:
            companies = list(best_gmvp["companies"])
            gmvp = best_gmvp["gmvp"]
            mrr = best_gmvp["mrr"]
            gmvp_df = full_price_df[["date"] + companies].copy()
            gmvp_df["date"] = pd.to_datetime(gmvp_df["date"])
            gmvp_df = gmvp_df.sort_values("date")
            sheet_name = "GMVP"
            gmvp_df.to_excel(writer, sheet_name=sheet_name, index=False)
            worksheet = writer.sheets[sheet_name]
            start_row = len(gmvp_df) + 3
            # Write weights
            worksheet.write(start_row, 0, "GMVP Weights")
            for i, (c, w) in enumerate(zip(companies, gmvp["weights"])):
                worksheet.write(start_row + i + 1, 0, c)
                worksheet.write(start_row + i + 1, 1, float(w))
            # Performance metrics
            gmvp_return_daily = gmvp["return"]
            gmvp_risk_daily = gmvp["risk"]
            gmvp_return_annual = gmvp_return_daily * TRADING_DAYS
            gmvp_risk_annual = gmvp_risk_daily * np.sqrt(TRADING_DAYS)
            metrics_row = start_row + len(companies) + 3
            worksheet.write(metrics_row, 0, "Performance Metrics")
            worksheet.write(metrics_row + 1, 0, "Daily Return")
            worksheet.write(metrics_row + 1, 1, float(gmvp_return_daily))
            worksheet.write(metrics_row + 2, 0, "Daily Risk")
            worksheet.write(metrics_row + 2, 1, float(gmvp_risk_daily))
            worksheet.write(metrics_row + 3, 0, "Annual Return")
            worksheet.write(metrics_row + 3, 1, float(gmvp_return_annual))
            worksheet.write(metrics_row + 4, 0, "Annual Risk")
            worksheet.write(metrics_row + 4, 1, float(gmvp_risk_annual))
        # ================= MRR =================
        if best_mrr is not None:
            companies = list(best_mrr["companies"])
            gmvp = best_mrr["gmvp"]
            mrr = best_mrr["mrr"]
            mrr_df = full_price_df[["date"] + companies].copy()
            mrr_df["date"] = pd.to_datetime(mrr_df["date"])
            mrr_df = mrr_df.sort_values("date")
            sheet_name = "MRR"
            mrr_df.to_excel(writer, sheet_name=sheet_name, index=False)
            worksheet = writer.sheets[sheet_name]
            start_row = len(mrr_df) + 3
            # Write weights
            worksheet.write(start_row, 0, "MRR Weights")
            for i, (c, w) in enumerate(zip(companies, mrr["weights"])):
                worksheet.write(start_row + i + 1, 0, c)
                worksheet.write(start_row + i + 1, 1, float(w))
            # Performance metrics
            mrr_return_daily = mrr["return"]
            mrr_risk_daily = mrr["risk"]
            mrr_return_annual = mrr_return_daily * TRADING_DAYS
            mrr_risk_annual = mrr_risk_daily * np.sqrt(TRADING_DAYS)
            sharpe_daily = mrr["sharpe"]
            sharpe_annual = sharpe_daily * np.sqrt(TRADING_DAYS)
            metrics_row = start_row + len(companies) + 3
            worksheet.write(metrics_row, 0, "Performance Metrics")
            worksheet.write(metrics_row + 1, 0, "Daily Return")
            worksheet.write(metrics_row + 1, 1, float(mrr_return_daily))
            worksheet.write(metrics_row + 2, 0, "Daily Risk")
            worksheet.write(metrics_row + 2, 1, float(mrr_risk_daily))
            worksheet.write(metrics_row + 3, 0, "Annual Return")
            worksheet.write(metrics_row + 3, 1, float(mrr_return_annual))
            worksheet.write(metrics_row + 4, 0, "Annual Risk")
            worksheet.write(metrics_row + 4, 1, float(mrr_risk_annual))
            worksheet.write(metrics_row + 5, 0, "Daily Sharpe")
            worksheet.write(metrics_row + 5, 1, float(sharpe_daily))
            worksheet.write(metrics_row + 6, 0, "Annual Sharpe")
            worksheet.write(metrics_row + 6, 1, float(sharpe_annual))
    print(f"\nBoth portfolios with full results exported to: {file_name}")

def export_selected_companies_to_excel(
    full_price_df,
    company_keys,
    file_name="selected_companies.xlsx",
    start_date=None,
    end_date=None,
    dropna=True
):
    """
    Exports selected company stock prices to Excel in format:

    date | Company1 | Company2 | ...

    PARAMETERS:
    full_price_df : loaded dataset
    company_keys  : list of company symbols
    file_name     : output Excel file name
    start_date    : optional filter
    end_date      : optional filter
    dropna        : remove columns with NaNs
    """
    if "date" not in full_price_df.columns:
        raise ValueError("DataFrame must contain 'date' column")
    df = full_price_df.copy()
    df["date"] = pd.to_datetime(df["date"])
    # Check which symbols exist
    available = set(df.columns)
    valid_companies = [c for c in company_keys if c in available]
    missing = [c for c in company_keys if c not in available]
    if missing:
        print("Warning: These companies were not found:")
        print(missing)
    if not valid_companies:
        raise ValueError("None of the requested companies exist in dataset")
    # Select columns
    df = df[["date"] + valid_companies]
    # Date filtering
    if start_date:
        df = df[df["date"] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df["date"] <= pd.to_datetime(end_date)]
    df = df.sort_values("date").reset_index(drop=True)
    if dropna:
        df = df.dropna(axis=1, how="any")
    # Export to Excel
    df.to_excel(file_name, index=False)
    print(f"\nExcel file created: {file_name}")
    print(f"Columns exported: {['date'] + valid_companies}")
    print(f"Total rows: {len(df)}")

def data_loading_pipeline_1():
    file_id = "1yJuY2SF3sCllibGXeyoNNzS6v2j7g08s"
    root = download_and_extract_google_drive_zip_1(file_id)
    df = read_stock_data_from_folder(root)
    print(df.head())

    available = set(df.columns)
    existing_codes = symbol_exists_checker(available, CHELSEA01); # NIFTY50_SYMBOLS
    print(existing_codes)
    filtered = filter_dataset(df, start_date="2018-01-01") #company_codes=existing_codes)
    print(filtered.head())
    print(filtered["date"].min())
    print(filtered["date"].max())
    return filtered

def data_loading_pipeline_2():
    file_id = "1edQni7obD5aQqfgqh_-bgz0NOOj6tXG7"
    root = download_and_extract_google_drive_zip_2(file_id)
    df = read_bse_bhavcopy_folder(root)
    #df = read_bse_bhavcopy_folder(root, strategy="top_turnover", strategy_params={"top_n": 1000}) # Top n by turnover # CONFIGURE HERE
    #df = read_bse_bhavcopy_folder(root, strategy="top_turnover", strategy_params={"top_n": 120}) # Random n
    #df = read_bse_bhavcopy_folder(root, strategy="price_filter", strategy_params={"min_price": 100}) # Price filter to remove penny stocks
    #df = read_bse_bhavcopy_folder(root, strategy="turnover_price", strategy_params={"top_n": 150, "min_price": 75})

    available = set(df.columns)
    existing_codes = symbol_exists_checker(available, CHELSEA01)  # CONFIGURE HERE

    filtered = filter_dataset(
      df,
      start_date="2024-08-01",
      company_codes=existing_codes # CONFIGURE HERE
    )

    print(filtered.head())
    print(filtered["date"].min())
    print(filtered["date"].max())
    return filtered

def main():

    # data = data_loading_pipeline_1() # for older dataset
    data = data_loading_pipeline_2() # for newer dataset

    #best_gmvp, best_mrr = BRUTE_FORCE_find_best_portfolios_of_size_x(data, x=3, risk_free_rate=RISK_FREE_RATE, allow_short=False)
    best_gmvp, best_mrr = BEAM_SEARCH_find_best_portfolios_of_size_x(data, x=10, risk_free_rate=RISK_FREE_RATE, allow_short=False) # CONFIGURE HERE

    print_portfolio("BEST GMVP PORTFOLIO", best_gmvp)
    print_portfolio("BEST MRR PORTFOLIO", best_mrr)
    # export_both_portfolios_to_excel(data, best_gmvp, best_mrr)

    export_selected_companies_to_excel(
      data,
      #["VAGHANI", "RRP", "EUROASIA", "MIDWEST", "GLITTEKG", "IMEC"],
      UTKARSH_TICKERS,
      file_name="best_mrr3.xlsx",
      start_date="2024-08-01"
    )

    # data = {
    #     "date": [
    #         "2020-01",
    #         "2020-02",
    #         "2020-03",
    #         "2020-04",
    #         "2020-05",
    #         "2020-06"
    #     ],
    #     "AAPL": [100, 105, 110, 108, 115, 120],
    #     "MSFT": [200, 210, 220, 215, 225, 230],
    #     "GOOG": [300, 295, 310, 320, 330, 340],
    #     "AMZN": [400, 420, 430, 440, 450, 460]
    # }

    # df = pd.DataFrame(data)
    # print("\nOriginal dataset:")
    # print(df)

    # filtered = filter_dataset(df, start_date="2020-02", end_date="2020-06", company_codes=["AAPL", "MSFT", "GOOG"])
    # print("\nFiltered dataset:\n", filtered)

    # returns = compute_returns(filtered)
    # print("\nReturns:\n", returns)

    # results = portfolio_optimizer(returns, risk_free_rate=0.01)
    # optimized_portfolio_printer(results)

if __name__ == "__main__":
    main()

Downloading...
From (original): https://drive.google.com/uc?id=1edQni7obD5aQqfgqh_-bgz0NOOj6tXG7
From (redirected): https://drive.google.com/uc?id=1edQni7obD5aQqfgqh_-bgz0NOOj6tXG7&confirm=t&uuid=ad1652db-1225-4a71-8abd-3aaddcef539b
To: /Users/utkarsh_kaushik/practice/python/IAPMAssignmentGroup/bse_data/stock_data_24_26_25Feb26.zip
100%|██████████| 89.3M/89.3M [00:05<00:00, 17.8MB/s]


Extracting ZIP...
Extraction complete.
Scanning CSV files...
Strategy: None (returning ALL stocks)
Strategy: None
Selected 9163 stocks
Final dataset shape: (363, 9164)
Available: 33
Missing: 9

Missing symbols:
['ASLIND', 'AXISILVER', 'KAPSTON', 'EGOLD', 'KRISHANA', 'CMMIPL', 'GVT&D', 'UNIHEALTH', 'VMARCIND']
symbol       date   TAKE  BANCOINDIA  KESORAMIND     BEL  LOTUSEYE  CRAFTSMAN  \
0      2024-08-01  19.81      744.00      218.60  311.15     67.03    5542.25   
1      2024-08-02  19.68      728.90      213.95  302.95     66.02    5354.00   
2      2024-08-05  18.99      680.65      209.95  290.10     64.06    5184.25   
3      2024-08-06  18.46      669.50      210.00  287.30     63.11    5176.75   
4      2024-08-07  19.73      683.80      212.85  300.25     67.88    5196.65   

symbol     CUB  QGOLDHALF  SILVER  ...     LTF  ABINFRA  UNIONBANK  BANKINDIA  \
0       171.95      58.77   85.33  ...  176.55      NaN     135.20     126.05   
1       166.85      59.35   85.29  ...  

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize


In [ ]:
# ============================================================
# SECTOR MAP
# ============================================================

SECTOR_MAP: dict[str, str] = {
    # Defense
    "BEL": "Defense", "HAL": "Defense", "GRSE": "Defense", "COCHINSHIP": "Defense",
    # Banking
    "SBIN": "Banking", "HDFCBANK": "Banking", "ICICIBANK": "Banking",
    "AXISBANK": "Banking", "KOTAKBANK": "Banking", "INDUSINDBK": "Banking",
    "BANKBARODA": "Banking", "UNIONBANK": "Banking", "BANKINDIA": "Banking",
    "DCBBANK": "Banking", "KARURVYSYA": "Banking", "CUB": "Banking",
    # IT
    "TCS": "IT", "INFY": "IT", "WIPRO": "IT", "HCLTECH": "IT", "TECHM": "IT",
    "LTIM": "IT", "NETWEB": "IT",
    # Oil & Gas
    "RELIANCE": "OilGas", "ONGC": "OilGas", "BPCL": "OilGas", "MRPL": "OilGas",
    # Pharma
    "SUNPHARMA": "Pharma", "DRREDDY": "Pharma", "CIPLA": "Pharma",
    "DIVISLAB": "Pharma", "APOLLOHOSP": "Pharma",
    # Auto
    "MARUTI": "Auto", "TATAMOTORS": "Auto", "BAJAJ-AUTO": "Auto",
    "EICHERMOT": "Auto", "HEROMOTOCO": "Auto", "M&M": "Auto",
    # FMCG
    "HINDUNILVR": "FMCG", "ITC": "FMCG", "BRITANNIA": "FMCG",
    "DABUR": "FMCG", "NESTLEIND": "FMCG", "TATACONSUM": "FMCG",
    # Finance (NBFC)
    "BAJFINANCE": "Finance", "BAJAJFINSV": "Finance", "SBILIFE": "Finance",
    "HDFCLIFE": "Finance", "LTF": "Finance", "TFCILTD": "Finance",
    # Metals & Mining
    "TATASTEEL": "Metals", "HINDALCO": "Metals", "JSWSTEEL": "Metals",
    "COALINDIA": "Metals", "ADANIENT": "Metals",
    # Power & Utilities
    "POWERGRID": "Power", "NTPC": "Power", "BGRENERGY": "Power",
    # Infrastructure
    "LT": "Infrastructure", "ADANIPORTS": "Infrastructure", "ABINFRA": "Infrastructure",
    # Commodities
    "ASIANPAINT": "Commodities", "ULTRACEMCO": "Commodities", "GRASIM": "Commodities",
}


# ============================================================
# MULTI-FACTOR STOCK SELECTION ENGINE
# ============================================================

def multi_factor_stock_selector(
    price_df: pd.DataFrame,
    top_n: int = 50,
    weights: dict | None = None
) -> list[str]:
    """
    Ranks stocks using multiple factors and returns the top N.

    Parameters
    ----------
    price_df : pd.DataFrame
        DataFrame with 'date' column and stock price columns.
    top_n : int
        Number of top stocks to return.
    weights : dict or None
        Factor weights with keys: 'momentum', 'low_volatility',
        'sharpe', 'sortino', 'max_drawdown'.
        Defaults to equal weights (0.2 each).

    Returns
    -------
    list[str]
        Symbols of top N stocks ranked by composite score.
    """
    if weights is None:
        weights = {
            "momentum": 0.2,
            "low_volatility": 0.2,
            "sharpe": 0.2,
            "sortino": 0.2,
            "max_drawdown": 0.2,
        }
    prices = price_df.copy()
    prices.set_index("date", inplace=True)
    prices.index = pd.to_datetime(prices.index)
    prices = prices.dropna(axis=1, how="any")
    returns = prices.pct_change().dropna()
    if returns.empty or returns.shape[1] == 0:
        return []

    # Factor 1: Momentum (cumulative return over the period)
    momentum = (1 + returns).prod() - 1

    # Factor 2: Low Volatility (lower std is better — negated in scoring)
    volatility = returns.std()

    # Factor 3: Sharpe Ratio
    mean_ret = returns.mean()
    sharpe = mean_ret / volatility

    # Factor 4: Sortino Ratio (downside risk-adjusted return)
    downside_std = returns.clip(upper=0).std()
    sortino = mean_ret / downside_std.replace(0, np.nan)

    # Factor 5: Maximum Drawdown (less negative is better — negated in scoring)
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.cummax()
    max_drawdown = ((cumulative - rolling_max) / rolling_max).min()

    def zscore(series: pd.Series) -> pd.Series:
        std = series.std()
        if std == 0:
            return series * 0
        return (series - series.mean()) / std

    composite_score = (
        weights["momentum"] * zscore(momentum)
        + weights["low_volatility"] * zscore(-volatility)
        + weights["sharpe"] * zscore(sharpe)
        + weights["sortino"] * zscore(sortino.fillna(0))
        + weights["max_drawdown"] * zscore(-max_drawdown)
    )
    return composite_score.nlargest(top_n).index.tolist()


# ============================================================
# SECTOR DIVERSIFICATION CONSTRAINT
# ============================================================

def sector_diversified_selection(
    symbols: list[str],
    sector_map: dict[str, str],
    max_per_sector: int = 3
) -> list[str]:
    """
    Filters a ranked list of symbols to ensure sector diversification.

    Parameters
    ----------
    symbols : list[str]
        Ordered list of stock symbols (highest-ranked first).
    sector_map : dict[str, str]
        Mapping from symbol to sector name (e.g., SECTOR_MAP).
    max_per_sector : int
        Maximum number of stocks allowed per sector.

    Returns
    -------
    list[str]
        Subset of symbols with at most max_per_sector per sector.
    """
    sector_count: dict[str, int] = {}
    selected: list[str] = []
    for symbol in symbols:
        sector = sector_map.get(symbol, "Unknown")
        count = sector_count.get(sector, 0)
        if count < max_per_sector:
            selected.append(symbol)
            sector_count[sector] = count + 1
    return selected


# ============================================================
# MARKET REGIME DETECTION
# ============================================================

def detect_market_regime(
    price_df: pd.DataFrame,
    index_col: str | None = None,
    short_window: int = 50,
    long_window: int = 200
) -> str:
    """
    Classifies the market as BULL, BEAR, or SIDEWAYS using moving averages.

    If the short MA is more than 2% above the long MA → BULL.
    If the short MA is more than 2% below the long MA → BEAR.
    Otherwise → SIDEWAYS.

    Parameters
    ----------
    price_df : pd.DataFrame
        Price DataFrame with 'date' column and stock price columns.
    index_col : str or None
        Column to use as the market index. Uses the first stock column if None.
    short_window : int
        Short moving average window in trading days.
    long_window : int
        Long moving average window in trading days.

    Returns
    -------
    str
        'BULL', 'BEAR', or 'SIDEWAYS'.
    """
    prices = price_df.copy()
    prices.set_index("date", inplace=True)
    prices.index = pd.to_datetime(prices.index)
    if index_col is not None and index_col in prices.columns:
        series = prices[index_col].dropna()
    else:
        series = prices.iloc[:, 0].dropna()
    if len(series) < long_window:
        return "SIDEWAYS"
    short_ma = series.rolling(window=short_window).mean()
    long_ma = series.rolling(window=long_window).mean()
    last_short = short_ma.iloc[-1]
    last_long = long_ma.iloc[-1]
    if pd.isna(last_short) or pd.isna(last_long) or last_long == 0:
        return "SIDEWAYS"
    ratio = (last_short - last_long) / last_long
    if ratio > 0.02:
        return "BULL"
    elif ratio < -0.02:
        return "BEAR"
    return "SIDEWAYS"


def regime_adjusted_weights(regime: str) -> dict[str, float]:
    """
    Returns multi-factor weights tuned for the detected market regime.

    Parameters
    ----------
    regime : str
        Market regime string: 'BULL', 'BEAR', or 'SIDEWAYS'.

    Returns
    -------
    dict[str, float]
        Weight dictionary compatible with multi_factor_stock_selector.
    """
    if regime == "BULL":
        return {
            "momentum": 0.35,
            "low_volatility": 0.10,
            "sharpe": 0.25,
            "sortino": 0.20,
            "max_drawdown": 0.10,
        }
    elif regime == "BEAR":
        return {
            "momentum": 0.05,
            "low_volatility": 0.35,
            "sharpe": 0.20,
            "sortino": 0.15,
            "max_drawdown": 0.25,
        }
    # SIDEWAYS
    return {
        "momentum": 0.20,
        "low_volatility": 0.20,
        "sharpe": 0.25,
        "sortino": 0.20,
        "max_drawdown": 0.15,
    }


In [ ]:
# ============================================================
# GENETIC ALGORITHM PORTFOLIO OPTIMIZER
# ============================================================

def genetic_algorithm_portfolio_search(
    filtered_df: pd.DataFrame,
    x: int,
    risk_free_rate: float = 0.0,
    allow_short: bool = True,
    population_size: int = 50,
    generations: int = 30,
    mutation_rate: float = 0.1,
    elite_size: int = 5
) -> tuple:
    """
    Genetic Algorithm portfolio optimizer.

    Uses stock combinations as chromosomes. Fitness function is Sharpe ratio
    (for MRR) and negative risk (for GMVP). Applies crossover and mutation
    operators across generations.

    Parameters
    ----------
    filtered_df : pd.DataFrame
        Price DataFrame with 'date' column and stock price columns.
    x : int
        Portfolio size (number of stocks per chromosome).
    risk_free_rate : float
        Daily risk-free rate used in Sharpe calculation.
    allow_short : bool
        Whether to allow short selling.
    population_size : int
        Number of chromosomes per generation.
    generations : int
        Number of generations to evolve.
    mutation_rate : float
        Probability of mutating one gene per chromosome after crossover.
    elite_size : int
        Number of best chromosomes preserved unchanged each generation.

    Returns
    -------
    tuple
        (best_gmvp, best_mrr) portfolio dictionaries.
    """
    companies = list(filtered_df.columns)
    companies.remove("date")
    if len(companies) < x:
        raise ValueError(f"Need at least {x} companies, got {len(companies)}")

    def _evaluate(combo: tuple) -> tuple:
        """Returns (gmvp_risk, mrr_sharpe, results_or_None)."""
        sub_df = filter_dataset(filtered_df, company_codes=list(combo))
        returns = compute_returns(sub_df)
        if returns.shape[1] < x:
            return np.inf, -np.inf, None
        Sigma = np.cov(returns.values, rowvar=False)
        if np.ndim(Sigma) >= 2 and np.linalg.cond(Sigma) > 1e12:
            return np.inf, -np.inf, None
        results = portfolio_optimizer(returns, risk_free_rate, allow_short)
        return results["gmvp"]["risk"], results["mrr"]["sharpe"], results

    # Initialise population with random unique-stock combinations
    population = [
        tuple(np.random.choice(companies, x, replace=False))
        for _ in range(population_size)
    ]

    best_gmvp = None
    best_gmvp_risk = np.inf
    best_mrr = None
    best_mrr_sharpe = -np.inf

    print(f"\n===== Genetic Algorithm: {generations} generations, population={population_size} =====")
    for gen in range(generations):
        fitness_list = []
        for chrom in population:
            gf, mf, res = _evaluate(chrom)
            fitness_list.append((chrom, gf, mf, res))
            if res is not None:
                if gf < best_gmvp_risk:
                    best_gmvp_risk = gf
                    best_gmvp = {"companies": chrom, "gmvp": res["gmvp"], "mrr": res["mrr"]}
                if mf > best_mrr_sharpe:
                    best_mrr_sharpe = mf
                    best_mrr = {"companies": chrom, "gmvp": res["gmvp"], "mrr": res["mrr"]}

        # Sort by MRR Sharpe (descending) for selection
        fitness_list.sort(key=lambda t: t[2], reverse=True)
        print(
            f"Gen {gen + 1}/{generations} | "
            f"Best GMVP risk={best_gmvp_risk:.6f} | "
            f"Best MRR Sharpe={best_mrr_sharpe:.6f}"
        )

        # Elitism: top chromosomes pass unchanged
        new_population = [t[0] for t in fitness_list[:elite_size]]

        # Selection pool: top half of population
        pool = [t[0] for t in fitness_list[:max(population_size // 2, elite_size)]]

        # Crossover + Mutation to fill remainder of population
        while len(new_population) < population_size:
            p1 = pool[np.random.randint(len(pool))]
            p2 = pool[np.random.randint(len(pool))]
            # Crossover: merge p1 and p2 genes, trim to size x
            child: list[str] = list(p1)
            for gene in p2:
                if gene not in child:
                    child.append(gene)
            if len(child) < x:
                remaining = [c for c in companies if c not in child]
                np.random.shuffle(remaining)
                child += remaining[:x - len(child)]
            child = child[:x]
            # Mutation: randomly replace one gene
            if np.random.random() < mutation_rate:
                idx = np.random.randint(x)
                candidates = [c for c in companies if c not in child]
                if candidates:
                    child[idx] = np.random.choice(candidates)
            new_population.append(tuple(child))

        population = new_population

    print("\n===== GA Complete =====")
    print(f"Best GMVP risk: {best_gmvp_risk:.6f}")
    print(f"Best MRR Sharpe: {best_mrr_sharpe:.6f}")
    return best_gmvp, best_mrr


In [ ]:
# ============================================================
# RISK PARITY PORTFOLIO
# ============================================================

def risk_parity_portfolio(returns_df: pd.DataFrame) -> dict:
    """
    Calculates a risk parity portfolio where each asset contributes
    equally to total portfolio risk.

    Uses scipy.optimize.minimize with the SLSQP method under long-only
    and full-investment constraints.

    Parameters
    ----------
    returns_df : pd.DataFrame
        Returns DataFrame with assets as columns and time as rows.

    Returns
    -------
    dict
        Dictionary with keys 'weights' (np.ndarray), 'return' (float),
        and 'risk' (float).
    """
    returns = returns_df.values
    n = returns.shape[1]
    Sigma = np.cov(returns, rowvar=False)

    if n == 1:
        return {
            "weights": np.array([1.0]),
            "return": float(returns.mean()),
            "risk": float(returns.std()),
        }

    def _risk_contributions(w: np.ndarray) -> np.ndarray:
        portfolio_var = float(w @ Sigma @ w)
        if portfolio_var <= 0:
            return np.zeros(n)
        return w * (Sigma @ w) / portfolio_var

    def _objective(w: np.ndarray) -> float:
        rc = _risk_contributions(w)
        target = 1.0 / n
        return float(np.sum((rc - target) ** 2))

    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]
    bounds = [(0.0, 1.0)] * n
    w0 = np.ones(n) / n

    result = minimize(
        _objective,
        w0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"ftol": 1e-10, "maxiter": 500},
    )

    weights = np.maximum(result.x, 0)
    weights /= weights.sum()
    mu = np.mean(returns, axis=0)
    port_return = float(weights @ mu)
    port_risk = float(np.sqrt(weights @ Sigma @ weights))
    return {"weights": weights, "return": port_return, "risk": port_risk}


In [ ]:
# ============================================================
# VISUALIZATION UTILITIES
# ============================================================

def plot_efficient_frontier(
    returns_df: pd.DataFrame,
    n_portfolios: int = 3000,
    risk_free_rate: float = 0.0,
    title: str = "Efficient Frontier"
) -> None:
    """
    Plots the efficient frontier by simulating random long-only portfolios.

    Each simulated portfolio is coloured by its annualised Sharpe ratio.

    Parameters
    ----------
    returns_df : pd.DataFrame
        Returns DataFrame with assets as columns and time as rows.
    n_portfolios : int
        Number of random portfolios to simulate.
    risk_free_rate : float
        Daily risk-free rate for Sharpe ratio calculation.
    title : str
        Title displayed on the chart.
    """
    returns = returns_df.values
    n = returns.shape[1]
    mu = np.mean(returns, axis=0)
    Sigma = np.cov(returns, rowvar=False)

    port_returns = []
    port_risks = []
    port_sharpes = []
    for _ in range(n_portfolios):
        w = np.random.dirichlet(np.ones(n))
        r = float(w @ mu)
        risk = float(np.sqrt(w @ Sigma @ w))
        sharpe = (r - risk_free_rate) / risk if risk > 0 else 0.0
        port_returns.append(r)
        port_risks.append(risk)
        port_sharpes.append(sharpe)

    port_returns = np.array(port_returns)
    port_risks = np.array(port_risks)
    port_sharpes = np.array(port_sharpes)

    plt.figure(figsize=(10, 6))
    sc = plt.scatter(
        port_risks * np.sqrt(TRADING_DAYS) * 100,
        port_returns * TRADING_DAYS * 100,
        c=port_sharpes * np.sqrt(TRADING_DAYS),
        cmap="viridis",
        alpha=0.5,
        s=10,
    )
    plt.colorbar(sc, label="Annual Sharpe Ratio")
    plt.xlabel("Annual Risk (%)")
    plt.ylabel("Annual Return (%)")
    plt.title(title)
    plt.tight_layout()
    plt.show()


def plot_portfolio_weights(
    portfolio: dict,
    title: str = "Portfolio Allocation"
) -> None:
    """
    Plots a grouped bar chart of GMVP and MRR weights for a portfolio.

    Parameters
    ----------
    portfolio : dict
        Portfolio dictionary with 'companies', 'gmvp', and 'mrr' keys
        (as returned by BEAM_SEARCH_find_best_portfolios_of_size_x,
        genetic_algorithm_portfolio_search, etc.).
    title : str
        Title displayed on the chart.
    """
    companies = list(portfolio["companies"])
    gmvp_weights = portfolio["gmvp"]["weights"]
    mrr_weights = portfolio["mrr"]["weights"]

    x = np.arange(len(companies))
    width = 0.35

    fig, ax = plt.subplots(figsize=(min(max(8, len(companies)), 20), 5))
    ax.bar(x - width / 2, gmvp_weights * 100, width, label="GMVP", color="steelblue")
    ax.bar(x + width / 2, mrr_weights * 100, width, label="MRR (Max Sharpe)", color="darkorange")
    ax.set_xticks(x)
    ax.set_xticklabels(companies, rotation=45, ha="right")
    ax.set_ylabel("Weight (%)")
    ax.set_title(title)
    ax.legend()
    ax.axhline(0, color="black", linewidth=0.8)
    plt.tight_layout()
    plt.show()


In [ ]:
# ============================================================
# EXAMPLE USAGE OF NEW FEATURES
# ============================================================

# --- Multi-Factor Selection (regime-aware) ---
# regime = detect_market_regime(data)                          # e.g. 'BULL'
# factor_weights = regime_adjusted_weights(regime)
# top_stocks = multi_factor_stock_selector(data, top_n=50, weights=factor_weights)
# diversified = sector_diversified_selection(top_stocks, SECTOR_MAP, max_per_sector=3)
# print(f"Regime: {regime}")
# print(f"Selected {len(diversified)} sector-diversified stocks:", diversified)

# --- Genetic Algorithm Optimizer ---
# filtered = filter_dataset(data, start_date="2024-08-01", company_codes=diversified)
# best_gmvp_ga, best_mrr_ga = genetic_algorithm_portfolio_search(
#     filtered, x=5, risk_free_rate=RISK_FREE_RATE, allow_short=False,
#     population_size=40, generations=20, mutation_rate=0.15
# )
# print_portfolio("GA BEST GMVP", best_gmvp_ga)
# print_portfolio("GA BEST MRR",  best_mrr_ga)

# --- Risk Parity Portfolio ---
# some_stocks = ["SBIN", "TCS", "RELIANCE", "HDFCBANK", "INFY"]
# filtered_rp = filter_dataset(data, company_codes=some_stocks)
# returns_rp  = compute_returns(filtered_rp)
# rp = risk_parity_portfolio(returns_rp)
# print("Risk Parity weights:", dict(zip(some_stocks, rp["weights"])))

# --- Visualizations ---
# filtered_vis = filter_dataset(data, company_codes=some_stocks)
# returns_vis  = compute_returns(filtered_vis)
# plot_efficient_frontier(returns_vis, n_portfolios=2000, risk_free_rate=RISK_FREE_RATE)
# if best_gmvp_ga is not None:
#     plot_portfolio_weights(best_gmvp_ga, title="GA GMVP Portfolio Weights")
